# 라이브러리 임포트 및 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import cv2
import os
from google.colab import files
import pandas as pd
import numpy as np

import numpy as np
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

##Load npy file

In [3]:
keypoints = np.load("/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/keypoints.npy", allow_pickle=True)
labels = np.load("/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/labels.npy", allow_pickle=True)

# From keypoint to tabular data

In [7]:
def keypoint_flatten(keypoints, label):
  columns = []

  for i in range(25):
    columns.extend([f'y{i}', f'x{i}'])
  columns.append('label')
  ml_data = pd.DataFrame(columns = columns)


  for idx, keypoint in enumerate(keypoints):
    flattend_data = []

    try :
      for i in keypoint[0]:
        flattend_data.append(i[0])
        flattend_data.append(i[1])
      flattend_data.append(label[idx])
      ml_data.loc[idx] = flattend_data
    except :
      pass
  return ml_data

In [8]:
df = keypoint_flatten(keypoints, labels)

# 데이터 증강 by Flipping Keypoints

In [9]:
# 새로운 DataFrame 생성하여 추가할 데이터 저장
augmented_data = []

for idx, row in df.iterrows():
    augmented_row = {}
    for col_name, value in row.items():
        if col_name.startswith('x'):
            new_value = 1920 - value  # 1920에서 뺀 값으로 대체
            augmented_row[col_name] = new_value
        else:
            augmented_row[col_name] = value
    augmented_data.append(augmented_row)

# 새로운 데이터를 포함한 DataFrame 생성
augmented_df = pd.DataFrame(augmented_data)

# 원래 데이터와 확장된 데이터 결합
final_df = pd.concat([df, augmented_df], ignore_index=True)


# standardization

In [10]:
standardized_df = final_df.copy()

def standardization(value, mean, std):
  return (value - mean) / std

for idx, row in standardized_df.iterrows():
    x_coord = []
    y_coord = []
    for col_name, value in row.items():
        if col_name.startswith('x'):
          x_coord.append(value)
        elif col_name.startswith('y'):
          y_coord.append(value)

    x_mean = np.array(x_coord).mean()
    x_std = np.array(x_coord).std()
    y_mean = np.array(y_coord).mean()
    y_std = np.array(y_coord).std()

    for col_name, value in row.items():
        if col_name.startswith('x'):
          standardized_df.loc[idx, col_name] = standardization(value, x_mean, x_std)
        elif col_name.startswith('y'):
          standardized_df.loc[idx, col_name] = standardization(value, y_mean, y_std)

standardized_df[:5]

,y0,x0,y1,x1,y2,x2,y3,x3,y4,x4,...,x20,y21,x21,y22,x22,y23,x23,y24,x24,label
0,-1.211710,-0.084188,-1.273957,0.292913,-1.278465,-0.404937,-1.250055,0.818830,-1.259836,-0.874990,...,0.819882,1.361415,0.474975,1.148089,-0.759188,1.111227,-1.109168,1.014218,-0.625431,0.0
1,-1.229617,-0.876085,-1.280594,-0.789579,-1.283216,-0.837689,-1.249739,-0.399690,-1.266595,-0.083295,...,-0.739517,1.340933,0.114045,1.225158,1.167877,1.196825,1.233221,1.183299,2.159367,0.0
2,-1.223080,-0.091482,-1.283774,0.267598,-1.282121,-0.461548,-1.270756,0.805753,-1.270191,-1.026884,...,0.874417,1.348272,0.543388,1.169636,-0.758669,1.139483,-1.162166,1.039530,-0.803478,0.0
3,-1.244221,-0.738662,-1.294174,-0.586785,-1.299885,-0.698386,-1.261048,-0.019007,-1.263030,-0.087514,...,-1.150055,1.346590,-0.074402,1.202012,0.816008,1.173670,0.879507,1.189782,1.935486,0.0
4,-1.229815,-0.046309,-1.288162,0.285960,-1.285892,-0.375895,-1.269560,0.764366,-1.263482,-0.878845,...,0.800977,1.336187,0.507166,1.168678,-0.920245,1.138770,-1.293106,1.040810,-0.741703,0.0


In [11]:
x = standardized_df.drop(columns = 'label')
y = standardized_df['label']

# Final Model : RandomForest(ML) -> with tabular data

In [12]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, stratify = y, test_size = 0.15, random_state = 42)

In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [14]:
model.fit(train_x, train_y)

RandomForestClassifier()

In [15]:
model.score(valid_x, valid_y)

0.9652173913043478